In [1]:
from flask import Flask
from config import Config
from flask_sqlalchemy import SQLAlchemy, BaseQuery
from flask_sqlalchemy import orm
from flask_migrate import Migrate
import os
from datetime import datetime

attribute_mapped_collection = orm.collections.attribute_mapped_collection

basedir = "/home/mtx/git/ranklens/app/test"


class Config(object):
    SECRET_KEY = os.environ.get('SECRET_KEY') or 'you-will-never-guess'
    DEBUG = True
    SQLALCHEMY_DATABASE_URI = os.environ.get('DATABASE_URL') or \
        'sqlite:///' + os.path.join(basedir, 'test_many2many.db')
    SQLALCHEMY_TRACK_MODIFICATIONS = True

app = Flask(__name__)
app.config.from_object(Config)

db = SQLAlchemy(app)
migrate = Migrate(app, db)

In [2]:
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(64), index=True, unique=True)
    email = db.Column(db.String(120), index=True, unique=True)
    tags = db.relationship('Tag', back_populates='user', lazy='dynamic')
    
    def __repr__(self):
        return '<User {}>'.format(self.username)


In [3]:
tags = db.Table('tags',
                db.Column('tag_id', db.Integer, db.ForeignKey('tag.id')),
                db.Column('page_id', db.Integer, db.ForeignKey('page.id')),
                db.Column('user_id', db.Integer, db.ForeignKey('tag.user_id')),
                )


class Page(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String)
    tags = db.relationship('Tag', secondary=tags,
                           collection_class=attribute_mapped_collection(
                               'tag_key'),
                           back_populates="pages")

    def __repr__(self):
        # return '<Page {} Tag {}>'.format(self.name, self.tags.all())
        return '<Page {} Tag {}>'.format(self.name, self.tags)


class Tag(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String)
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'))
    pages = db.relationship('Page', secondary=tags, back_populates="tags")
    user = db.relationship('User', secondary=tags, back_populates='tags', lazy='dynamic')

    @property
    def tag_key(self):
        return self.user

    def __repr__(self):
        return '<Tag {} : {}>'.format(self.name, self.user)

In [4]:
def get_untagged_image(u):    
    tag_u = Tag.query.filter(Tag.user==u)
    myfilter=[~Page.tags.contains(i) for i in tag_u]
    return Page.query.filter(*myfilter).all()

In [5]:
db.drop_all()

In [6]:
db.create_all()

## create users, pages and tags

In [7]:
u = User(username='test', email='test@test.com')
u2 = User(username='test2', email='test2@test.com')
db.session.add(u)
db.session.add(u2)
db.session.commit()

In [8]:
User.query.all()

[<User test>, <User test2>]

In [10]:
t1 = Tag(name='A')
t1.user.append(u)
t2 = Tag(name='B')
t2.user.append(u)
t3 = Tag(name='C')
t3.user.append(u)
t4 = Tag(name='A')
t4.user.append(u2)
t5 = Tag(name='B')
t5.user.append(u2)
t6 = Tag(name='C')
t6.user.append(u2)
db.session.add_all([t1,t2,t3,t4,t5,t6])
db.session.commit()

In [11]:
p1 = Page(name='p1')
p2 = Page(name='p2')
db.session.add_all([p1,p2])
db.session.commit()

In [12]:
Tag.query.all()

[<Tag A : [<User test>]>,
 <Tag B : [<User test>]>,
 <Tag C : [<User test>]>,
 <Tag A : [<User test2>]>,
 <Tag B : [<User test2>]>,
 <Tag C : [<User test2>]>]

In [13]:
Page.query.all()

[<Page p1 Tag {}>, <Page p2 Tag {}>]

## add tag on pages

In [15]:
p1.tags[t1.user[0]]=t1
p1.tags[t2.user[0]]=t2
p2.tags[t2.user[0]]=t2
p2.tags[t4.user[0]]=t4

In [16]:
Page.query.all()

[<Page p1 Tag {<User test>: <Tag B : [<User test>]>}>,
 <Page p2 Tag {<User test>: <Tag B : [<User test>]>, <User test2>: <Tag A : [<User test2>]>}>]

## query test

In [17]:
Page.query.join(Page.tags).filter(Tag.name=="A").all()

[<Page p2 Tag {<User test>: <Tag B : [<User test>]>, <User test2>: <Tag A : [<User test2>]>}>]

In [18]:
Page.query.join(Page.tags).filter(Tag.name=="A").first().tags[u]

<Tag B : [<User test>]>

In [19]:
db.session.query(tags).all()

[(2, None, 1),
 (4, None, 2),
 (1, None, 1),
 (3, None, 1),
 (5, None, 2),
 (6, None, 2),
 (2, 2, None),
 (4, 2, None),
 (2, 1, None)]

### select un-tagged page under one user

In [18]:
Page.query.all()

[<Page p1 Tag {<User test>: <Tag B : <User test>>}>,
 <Page p2 Tag {<User test>: <Tag B : <User test>>, <User test2>: <Tag A : <User test2>>}>]

In [19]:
get_untagged_image(u2)

[<Page p1 Tag {<User test>: <Tag B : <User test>>}>]

In [20]:
get_untagged_image(u)

[]

In [18]:
s1 = Tag.query.filter(Tag.user==u2)
fr=[~Page.tags.contains(i) for i in s1]
Page.query.filter(*fr).all()

[<Page p1 Tag {<User test>: <Tag B : <User test>>}>]

In [19]:
s1 = Tag.query.filter(Tag.user==u)
fr=[~Page.tags.contains(i) for i in s1]
Page.query.filter(*fr).all()

[]

### select tag A pages under user 1

In [21]:
Page.query.join(Page.tags).filter(Tag.name=="B").filter(Tag.user==u).all()

[<Page p1 Tag {<User test>: <Tag B : <User test>>}>,
 <Page p2 Tag {<User test>: <Tag B : <User test>>, <User test2>: <Tag A : <User test2>>}>]

In [22]:
Page.query.join(Page.tags).filter(Tag.name=="A").filter(Tag.user==u2).all()

[<Page p2 Tag {<User test>: <Tag B : <User test>>, <User test2>: <Tag A : <User test2>>}>]

In [23]:
Page.query.join(Page.tags).filter(Tag.name=="B").filter(Tag.user==u2).all()

[]

In [24]:
t = Tag.query.filter(Tag.user_id==u.id).filter(Tag.name=='B').first()
t.pages.all()

[<Page p1 Tag {<User test>: <Tag B : <User test>>}>,
 <Page p2 Tag {<User test>: <Tag B : <User test>>, <User test2>: <Tag A : <User test2>>}>]

In [25]:
t = Tag.query.filter(Tag.user_id==u2.id).filter(Tag.name=='A').first()
t.pages.all()

[<Page p2 Tag {<User test>: <Tag B : <User test>>, <User test2>: <Tag A : <User test2>>}>]

In [26]:
t = Tag.query.filter(Tag.user_id==u2.id).filter(Tag.name=='B').first()
t.pages.all()

[]

In [19]:
Page.query.join(tags, (tags.c.tag_id == Page.tag_id))

AttributeError: type object 'Page' has no attribute 'tag_id'

In [18]:
tags.c.tag_id

Column('tag_id', Integer(), ForeignKey('tag.id'), table=<tags>)